In [1]:
import pandas as pd
from tqdm import tqdm
import time
import re
!pip install selenium
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

In [2]:
def login(driver,url):
    driver.get(url)
    driver.maximize_window()
    user=input("Enter your username: ")
    passw=input("Enter your password: ")
    username= WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name='username']")))
    password= WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name='password']")))
    username.clear()
    password.clear()
    username.send_keys(user)
    password.send_keys(passw)
    login= WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[type='submit']"))).click()
    login3= WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//button[contains(text(),'Not Now')]"))).click()

In [3]:
def get_handle(driver,url,handle):
    handle=handle+'/?hl=en'
    driver.get(url+handle)

In [4]:
def scrape(driver,nop):
    posts=WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//*[text()=' posts']")))
    posts=(posts.text).split()[0]
    posts=re.sub(',','',posts)
    if(nop.lower()=='all'):
        nop=int(posts)
        
    else:
        nop=int(nop)
    photo= WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"div[class='_aagw']"))).click()
    result_cap=[]
    result_likes=[]
    ptype=[]
    p_or_u=[]
    for i in tqdm(range(nop)):
        time.sleep(0.5)
        vid=driver.find_elements(By.TAG_NAME,'video')
        paid=(driver.find_elements(By.XPATH,"//*[text()='Paid partnership']")) or (driver.find_elements(By.XPATH,"//*[text()='Paid partnership with ']"))
        if(len(paid)==0):
            p_or_u.append('Not sponsored')
            
        else:
            p_or_u.append('Sponsored')
            
        if (len(vid)==0):
            ptype.append('Photo')
            
        else:
            ptype.append('Video')
        
        likes=WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//*[text()=' likes']")))
        likes=int(re.sub(',','',likes.text.split()[0]))
        result_likes.append(likes)
        try:
            cap=WebDriverWait(driver,1).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"h1[class='_aacl _aaco _aacu _aacx _aad7 _aade']")))
        except TimeoutException:
            result_cap.append("NO_CAPTION")
            photo= WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"div[class=' _aaqg _aaqh']"))).click()
            continue
        result_cap.append(cap.text)
        photo= WebDriverWait(driver,20).until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR,"div[class=' _aaqg _aaqh']"))).click()
    
    df=pd.DataFrame()
    for i in range(len(result_cap)):
        result_cap[i]=" ".join(re.sub('\n',' ',result_cap[i]).split())
    df['LIKES']=result_likes
    df['CAPTIONS']=result_cap
    df['TYPE']=ptype
    df['SPONSOR']=p_or_u
    return df

In [5]:
def find(string,keyword):
    string=string.lower()
    if keyword in string:
        return True
    
    else:
        return False

In [6]:
def find_word(df,kw):
    kw=kw.lower()
    df['FOUND']=df.apply(lambda i:find(i['CAPTIONS'],kw),axis=1)        
    return df[df['FOUND']==True]
    

In [ ]:
def main():
    handle=input("Enter the handle: ")
    keyword=input("Enter the keyword to be searched: ")
    no_of_posts=input("Enter number of posts you want to search: ")
    ques=input("Do you want excel?(Y/N): ")
    url="https://www.instagram.com/"
    driver=webdriver.Chrome()
    login(driver,url)
    get_handle(driver,url,handle)
    df=scrape(driver,no_of_posts)
    if(ques.lower()=='yes'):
        df.to_excel("instaposts.xlsx")
        
    driver.minimize_window()
    ans=find_word(df,keyword)
    print("The post with the keyword: ")
    print(ans)
    time.sleep(5)
    
if __name__==("__main__"):
    main()